In [5]:
import sys
sys.path.insert(0, '..')

import numpy as np
from activation_func.relu import Activation_ReLU
from layers.layer_dense import Layer_Dense

In [6]:
#max pooling over batch of images
def max_pool(inputs,f,stride):
    #get shapes
    (m,n_h_prev,n_w_prev,n_c_prev)=inputs.shape
    
    #define output shape
    n_h=int((n_h_prev-f)/stride)+1
    n_w=int((n_w_prev-f)/stride)+1
    
    #output
    output=np.zeros((m,n_h,n_w,n_c_prev))

    for i in range(m):
        inputi=inputs[i]
        for h in range(n_h):
            for w in range(n_w):
                for c in range(n_c_prev):
                    #define vert/horiz start-end  
                    vert_start=h*stride
                    vert_end=vert_start+f
                    horiz_start=w*stride
                    horiz_end=horiz_start+f
                    #get image slice 
                    inputi_slice=inputi[vert_start:vert_end,horiz_start:horiz_end,c]
                    #max pooling on inputs[i] slice
                    output[i,h,w,c]=np.max(inputi_slice)

    return output


In [7]:
inputs=np.random.rand(3,15,15,3)
#hyperparameters shrinking by a factor of 2 
f=2
stride=2
output=max_pool(inputs,f,stride)

print('shape of inputs before max-pool ',inputs.shape)
print('shape of inputs after max-pool ',output.shape)

shape of inputs before max-pool  (3, 15, 15, 3)
shape of inputs after max-pool  (3, 7, 7, 3)


In [8]:
#CNN example forward pass ||something similar to LeNet-5  
inputs=np.random.rand(5,32,32,3)

#layer1 
relu1=Activation_ReLU()
w1=np.random.rand(5,5,3,6)
b1=np.random.rand(1,1,1,6)
stride1=1
padding1=0
#forward pass layer1(convolution + relu + max pooling)
conv1_out=conv_forward(inputs,w1,b1,padding1,stride1)
relu1.forward(conv1_out,training=True)
max_pool1=max_pool(relu1.output,f=2,stride=2)

#layer2
relu2=Activation_ReLU()
w2=np.random.rand(5,5,6,16)
b2=np.random.rand(1,1,1,16)
stride2=1
padding2=0
#forward pass layer2(conv,relu,max-pool)
conv2_out=conv_forward(max_pool1,w2,b2,padding2,stride2)
relu2.forward(conv2_out,training=True)
max_pool2=max_pool(relu2.output,f=2,stride=2)


#flatten/reshape the max-pool2
x=max_pool2.reshape(max_pool2.shape[0],-1)

#fully-connected layer3
layer3=Layer_Dense(400,120)
relu3=Activation_ReLU()
#forward pass layer3(linear-layer,relu)
layer3.forward(x,training=True)
relu3.forward(layer3.output,training=True)

#fully-connected layer4
layer4=Layer_Dense(120,84)
relu4=Activation_ReLU()
layer4.forward(relu3.output,training=True)
relu4.forward(layer4.output,training=True)

#fully-connected layer5
layer5=Layer_Dense(84,10)
relu5=Activation_ReLU()
layer5.forward(relu4.output,training=True)
relu5.forward(layer5.output,training=True)


#softmax probability distribution 
softmax=Activation_Softmax()
softmax.forward(relu5.output,training=True)

print('shape of inputs',inputs.shape)
print('shape of 1st layer max-out',max_pool1.shape) 
print('shape of 2st layer max-out',max_pool2.shape)
print('shape of inputs flattened',x.shape)
print('shape of 3st fc relu',relu3.output.shape)
print('shape of 4th fc relu out',relu4.output.shape)
print('shape of 5th fc relu out',relu5.output.shape)
print('final softmax shape',softmax.output.shape)

NameError: name 'conv_forward' is not defined

In [9]:
import torch
import torch.nn as nn

input=torch.rand(1,1,3,3)
m=nn.Conv2d(1,2,[2,2],bias=True,padding=2,padding_mode='zeros',stride=2)
m.requires_grad_=True

out=m(input)

t=out.sum()

t.backward()

t_weights_grad=m.weight.grad.permute(2,3,1,0).numpy()[:,:,0,:]
t_bias=m.bias.grad
t_out=out


In [10]:
def zero_padding2d(inputs,padding):
    inputs_padded = np.pad(inputs, ((0,0), (padding,padding), (padding,padding),
                            ), mode='constant', constant_values = (0,0))
    return inputs_padded

In [11]:
#simple 2d convolution forward/backward pass  without biases  
inputs=input.permute(0,2,3,1).numpy()[:,:,:,0]
#permute the shapes to match [fh,fw,n_c_out] 
weights=m.weight.permute(2,3,1,0).detach().numpy()[:,:,0,:]
bias=m.bias.detach().numpy()
pad=2
stride=2


(m,n_h_prev,n_w_prev)=inputs.shape
(f_h,f_w,n_c)=weights.shape

n_h=int((n_h_prev-f_h+2*pad)/stride)+1
n_w=int((n_w_prev-f_w+2*pad)/stride)+1

output=np.zeros((m,n_h,n_w,n_c))

input_pad=zero_padding2d(inputs,pad)

#forward pass works 
for i in range(m):
    inputi=input_pad[i]
    for h in range(n_h):
        for w in range(n_w):
            for c in range(n_c):
                vert_start=h*stride
                vert_end=vert_start+f_h
                horiz_start=w*stride
                horiz_end=horiz_start+f_w

                input_slice=inputi[vert_start:vert_end,horiz_start:horiz_end]
                output[i,h,w,c]=np.sum(np.multiply(input_slice,weights[:,:,c]))+bias[c]

#backward pass 
dinputs=np.ones_like(output)
dw=np.zeros_like(weights)
db=np.sum(dinputs,axis=(0,1,2))

for i in range(m):
    inputi=input_pad[i]
    for h in range(n_h):
        for w in range(n_w):
            for c in range(n_c):
                vert_start=h*stride
                vert_end=vert_start+f_h
                horiz_start=w*stride
                horiz_end=horiz_start+f_w
                input_slice=inputi[vert_start:vert_end,horiz_start:horiz_end]
                dw[:,:,c]+=np.multiply(input_slice,dinputs[i,h,w,c])

#bias in this implementation expects batch of images 
print('dw grad ',dw)
print('db grad ',db)

dw grad  [[[1.1292248  1.1292248 ]
  [0.98203516 0.98203516]]

 [[0.49051005 0.49051005]
  [0.23187637 0.23187637]]]
db grad  [9. 9.]


In [13]:
#t=compare.trunc(t_out.permute(0,2,3,1).detach().numpy(),3)
#c=compare.trunc(output,3)
#todo 
np.allclose(t_out.permute(0,2,3,1).detach().numpy(),output)

True

In [60]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

In [61]:
np.equal(trunc(t_out.permute(0,2,3,1).detach().numpy(),3),trunc(output,3))

array([[[[False, False],
         [False, False],
         [False, False]],

        [[False, False],
         [False, False],
         [False, False]],

        [[False, False],
         [False, False],
         [False, False]]]])

In [30]:
from convolution.conv_2d import Conv2D

conv2d=Conv2D([1],1,padding=2,stride=2)
conv2d.set_params(weights,bias)

print(inputs.shape)
conv2d.forward(inputs)


conv2d.backward(dinputs)
#compare to numpy results above 
print('output compare',(output==conv2d.output).all())
print('dweights compare',(conv2d.dweights==dw).all())
print('dbiases compare',(conv2d.dbiases==db).all())

(1, 3, 3)
output compare True
dweights compare True
dbiases compare True


In [26]:
conv_dw_trun=trunc(conv2d.dweights,3)
t_dw_trunc=trunc(t_weights_grad,3)
(conv_dw_trun==t_dw_trunc).all()

AttributeError: 'bool' object has no attribute 'all'

In [311]:
#compare
dw_trunc=trunc(dw,3)
torch_dw_trunc=trunc(t_weights_grad,3)
#compare values 
(dw_trunc==torch_dw_trunc).all()


True